In [2]:
import os
import pandas as pd
import numpy as np
pd.set_option("display.max_colwidth", 2)

os.chdir("/home/coinse/Workspace/SBFL")

# Load bug info

In [3]:
from scripts.util import load_bug_data, find_covmat_func, find_covmat_idx
from functools import reduce
from typing import Set

pid, vid = "cppcheck", 25

bug = load_bug_data(pid, vid)

print(f"{bug.buggy_files=}")
print(f"{bug.buggy_lines=}")
print("bug.coverage_df=")
display(bug.coverage_df)
print(f"{bug.failing_tests=}")
print(f"{bug.failing_tests_GT=}")
print("bug.functions={", end="")
for func in list(bug.functions)[:5]:
    print(f"{func},", end=" ")
print(", ...}")
print(f"{bug.id=}")
print(f"{bug.num_test_cases=}")
print(f"{bug.passing_tests=}")
print(f"{bug.path_to_cov_df=}")
print(f"{bug.path_to_patch=}")
print(f"{bug.path_to_test_results=}")
print("bug.source_files=", end="")
for src in list(bug.source_files)[:5]:
    print(f"{src},", end=" ")
print(", ...}")

fault_idxs = sorted(bug.buggy_lines)
# fault_idx2meth = {}
fault_meth2idx = {}
err_msg = ""
for fault_idx in fault_idxs:
    succ_func, ret_func = find_covmat_func(
        pid, bug.coverage_df, fault_idx, debug=False
    )
    if succ_func:
        succ_idx, ret_idx = find_covmat_idx(
            pid, bug.coverage_df, fault_idx, debug=False
        )
        if succ_idx:
            if ret_func in fault_meth2idx:
                fault_meth2idx[ret_func].add(ret_idx)
            else:
                fault_meth2idx[ret_func] = {ret_idx}
        else:
            print(f"{ret_idx}")
            if ret_func not in fault_meth2idx:
                fault_meth2idx[ret_func] = set()
    else:
        print(f"{ret_func}")
# fault_meths: Set = reduce(
#     lambda x, y: x | set([y]), fault_idx2meth.values(), set()
# )
# fault_meth2idx = {
#     v: {idx for idx in fault_idx2meth if fault_idx2meth[idx] == v}
#     for v in fault_meths
# }
fault_meths = set(fault_meth2idx.keys())
for meth in fault_meth2idx:
    print(f"{meth=}: {fault_meth2idx[meth]}")
# print(f"{fault_meths=}")


bug.buggy_files={'lib/valueflow.cpp'}
bug.buggy_lines={('lib/valueflow.cpp', 3070), ('lib/valueflow.cpp', 3071)}
bug.coverage_df=


44  \
file                                     function                                                                                                                    line         
/home/workspace/lib/checkunusedvar.h     _ZN14CheckUnusedVarD2Ev                                                                                                     44    0      
                                         _ZN14CheckUnusedVarC2Ev                                                                                                     47    1      
                                                                                                                                                                     48    1      
                                         _ZN14CheckUnusedVarC2EPK9TokenizerPK8SettingsP11ErrorLogger                                                                 51    0      
                                                                                                                                                                     52    0      
...                                                                                                                                                                       ..      
/usr/include/c++/7/bits/refwrap.h        _ZSt3refIKNSt7__debug13unordered_mapIiPK8VariableSt4hashIiESt8equal_toIiESaISt4pairIKiS4_EEEEESt17reference_wrapperIT_ERSG_ 364   2570   
                                                                                                                                                                     365   2570   
/home/workspace/lib/templatesimplifier.h _ZN18TemplateSimplifier12TokenAndName5tokenEP5Token                                                                         166   0      
                                                                                                                                                                     167   0      
                                                                                                                                                                     168   0      

                                                                                                                                                                           42  \
file                                     function                                                                                                                    line       
/home/workspace/lib/checkunusedvar.h     _ZN14CheckUnusedVarD2Ev                                                                                                     44    0    
                                         _ZN14CheckUnusedVarC2Ev                                                                                                     47    1    
                                                                                                                                                                     48    1    
                                         _ZN14CheckUnusedVarC2EPK9TokenizerPK8SettingsP11ErrorLogger                                                                 51    0    
                                                                                                                                                                     52    0    
...                                                                                                                                                                       ..    
/usr/include/c++/7/bits/refwrap.h        _ZSt3refIKNSt7__debug13unordered_mapIiPK8VariableSt4hashIiESt8equal_toIiESaISt4pairIKiS4_EEEEESt17reference_wrapperIT_ERSG_ 364   68   
                                                                                                                                                                     365   68   
/home/workspace/lib/templatesimplifier.h _ZN18TemplateSimplifier12TokenAndN

bug.failing_tests={'5'}
bug.failing_tests_GT={'5'}
bug.functions={('/home/workspace/lib/checkcondition.cpp', '_ZN14CheckCondition15comparisonErrorEPK5TokenRKNSt7__cxx1112basic_stringIcSt11char_traitsIcESaIcEEExSA_xb'), ('/home/workspace/lib/library.cpp', '_ZNK7Library15returnValueTypeB5cxx11EPK5Token'), ('/home/workspace/lib/pathmatch.cpp', '_Z41__static_initialization_and_destruction_0ii'), ('/home/workspace/lib/astutils.cpp', '_ZL14getArgumentVarPK5Tokeni'), ('/home/workspace/lib/token.cpp', '_ZZNK5Token16hasKnownIntValueEvENKUlRKN9ValueFlow5ValueEE_clES3_'), , ...}
bug.id='cppcheck-25'
bug.num_test_cases=85
bug.passing_tests={'45', '7', '12', '54', '18', '20', '22', '80', '42', '76', '78', '11', '15', '74', '39', '53', '61', '25', '2', '9', '14', '68', '34', '26', '52', '23', '84', '55', '65', '72', '75', '71', '48', '77', '8', '49', '17', '10', '64', '30', '6', '82', '31', '60', '51', '27', '38', '66', '33', '32', '43', '41', '57', '81', '83', '62', '16', '19', '79', '67', '56', '5

# Get spectra

In [4]:
covmat = bug.coverage_df
pass_tcs = bug.passing_tests
fail_tcs = bug.failing_tests

covmat_bool = covmat.astype(bool)
P = len(pass_tcs)
F = len(fail_tcs)
e_p = covmat_bool[pass_tcs].sum(axis=1)
e_f = covmat_bool[fail_tcs].sum(axis=1)
n_p = P - e_p
n_f = F - e_f
spectra = pd.DataFrame(
    {
        "line": covmat.index.to_list(),
        "e_p": e_p,
        "e_f": e_f,
        "n_p": n_p,
        "n_f": n_f,
    }
)

display(spectra)


line  \
file                                     function                                                                                                                    line                                                                                                                                                                          
/home/workspace/lib/checkunusedvar.h     _ZN14CheckUnusedVarD2Ev                                                                                                     44    (/home/workspace/lib/checkunusedvar.h, _ZN14CheckUnusedVarD2Ev, 44)                                                                                                     
                                         _ZN14CheckUnusedVarC2Ev                                                                                                     47    (/home/workspace/lib/checkunusedvar.h, _ZN14CheckUnusedVarC2Ev, 47)                                                                                                     
                                                                                                                                                                     48    (/home/workspace/lib/checkunusedvar.h, _ZN14CheckUnusedVarC2Ev, 48)                                                                                                     
                                         _ZN14CheckUnusedVarC2EPK9TokenizerPK8SettingsP11ErrorLogger                                                                 51    (/home/workspace/lib/checkunusedvar.h, _ZN14CheckUnusedVarC2EPK9TokenizerPK8SettingsP11ErrorLogger, 51)                                                                 
                                                                                                                                                                     52    (/home/workspace/lib/checkunusedvar.h, _ZN14CheckUnusedVarC2EPK9TokenizerPK8SettingsP11ErrorLogger, 52)                                                                 
...                                                                                                                                                                                                                                                                            ...                                                                 
/usr/include/c++/7/bits/refwrap.h        _ZSt3refIKNSt7__debug13unordered_mapIiPK8VariableSt4hashIiESt8equal_toIiESaISt4pairIKiS4_EEEEESt17reference_wrapperIT_ERSG_ 364   (/usr/include/c++/7/bits/refwrap.h, _ZSt3refIKNSt7__debug13unordered_mapIiPK8VariableSt4hashIiESt8equal_toIiESaISt4pairIKiS4_EEEEESt17reference_wrapperIT_ERSG_, 364)   
                                                                                                                                                                     365   (/usr/include/c++/7/bits/refwrap.h, _ZSt3refIKNSt7__debug13unordered_mapIiPK8VariableSt4hashIiESt8equal_toIiESaISt4pairIKiS4_EEEEESt17reference_wrapperIT_ERSG_, 365)   
/home/workspace/lib/templatesimplifier.h _ZN18TemplateSimplifier12TokenAndName5tokenEP5Token                                                                         166   (/home/workspace/lib/templatesimplifier.h, _ZN18TemplateSimplifier12TokenAndName5tokenEP5Token, 166)                                                                    
                                                                                                                                                                     167   (/home/workspace/lib/templatesimplifier.h, _ZN18TemplateSimplifier12TokenAndName5tokenEP5Token, 167)                                                                    
                                                                                                                                                                     168   (/home/workspace/lib/templatesimplifier.h, _ZN18TemplateSimplifier12TokenAndName5

# Compute suspiciousness scores

In [5]:
def ochiai(e_f, e_p, n_f, n_p):
    return e_f / np.sqrt(e_p + e_f) / np.sqrt(e_f + n_f) if e_f else 0.0


def op2(e_f, e_p, n_f, n_p):
    return e_f - e_p / (e_p + n_p + 1)

spectra["Sop2"] = spectra.apply(
    lambda row: op2(row.e_f, row.e_p, row.n_f, row.n_p), axis=1
)
spectra["Sochiai"] = spectra.apply(
    lambda row: ochiai(row.e_f, row.e_p, row.n_f, row.n_p), axis=1
)
spectra["Rop2"] = spectra.Sop2.rank(ascending=False, method="max")
spectra["Rochiai"] = spectra.Sochiai.rank(ascending=False, method="max")


# Line level SBFL

In [6]:
for fault_meth in fault_meth2idx:
    fault_idxs_of_meth = fault_meth2idx[fault_meth]
    print("fault meth:", fault_meth)
    for fault_idx in fault_idxs_of_meth:
        print("fault idx:", tuple(fault_idx))
    print("SBFL score of faulty method:")
    display(spectra.loc[fault_idxs_of_meth].reset_index(drop=True))

print("Top 5 SBFL scores:")
print("Op2:")
display(
    spectra.sort_values("Sop2", ascending=False).head(5).reset_index(drop=True)
)
print("Ochiai:")
display(
    spectra.sort_values("Sochiai", ascending=False)
    .head(5)
    .reset_index(drop=True)
)


fault meth: ('/home/workspace/lib/valueflow.cpp', '_ZL18isLifetimeBorrowedPK9ValueTypeS1_')
fault idx: ('/home/workspace/lib/valueflow.cpp', '_ZL18isLifetimeBorrowedPK9ValueTypeS1_', 3070)
fault idx: ('/home/workspace/lib/valueflow.cpp', '_ZL18isLifetimeBorrowedPK9ValueTypeS1_', 3071)
SBFL score of faulty method:


,line,e_p,e_f,n_p,n_f,Sop2,Sochiai,Rop2,Rochiai
0,"(/home/workspace/lib/valueflow.cpp, _ZL18isLifetimeBorrowedPK9ValueTypeS1_, 3070)",18,1,66,0,0.788235,0.229416,2099.0,2099.0
1,"(/home/workspace/lib/valueflow.cpp, _ZL18isLifetimeBorrowedPK9ValueTypeS1_, 3071)",27,1,57,0,0.682353,0.188982,3601.0,3601.0


Top 5 SBFL scores:
Op2:


,line,e_p,e_f,n_p,n_f,Sop2,Sochiai,Rop2,Rochiai
0,"(/home/workspace/lib/valueflow.cpp, _ZNK13LifetimeStore11byDerefCopyIZNKS_11byDerefCopyEP5TokenP9TokenListP11ErrorLoggerPK8SettingsEUlPKS1_E_EEvS2_S4_S6_S9_T_, 3423)",0,1,84,0,1.0,1.0,156.0,156.0
1,"(/home/workspace/lib/checkautovariables.cpp, _ZL15isDeadTemporarybPK5TokenS1_PK7Library, 449)",0,1,84,0,1.0,1.0,156.0,156.0
2,"(/home/workspace/lib/astutils.cpp, _Z11isTemporarybPK5TokenPK7Libraryb, 240)",0,1,84,0,1.0,1.0,156.0,156.0
3,"(/home/workspace/lib/astutils.cpp, _Z11isTemporarybPK5TokenPK7Libraryb, 241)",0,1,84,0,1.0,1.0,156.0,156.0
4,"(/home/workspace/lib/checkautovariables.cpp, _ZL11isDeadScopePK5TokenPK5Scope, 433)",0,1,84,0,1.0,1.0,156.0,156.0


Ochiai:


,line,e_p,e_f,n_p,n_f,Sop2,Sochiai,Rop2,Rochiai
0,"(/home/workspace/lib/valueflow.cpp, _ZL25valueFlowLifetimeFunctionP5TokenP9TokenListP11ErrorLoggerPK8Settings, 3477)",0,1,84,0,1.0,1.0,156.0,156.0
1,"(/home/workspace/lib/astutils.cpp, _Z11isTemporarybPK5TokenPK7Libraryb, 273)",0,1,84,0,1.0,1.0,156.0,156.0
2,"(/home/workspace/lib/valueflow.cpp, _Z12lifetimeTypeB5cxx11PK5TokenPKN9ValueFlow5ValueE, 2833)",0,1,84,0,1.0,1.0,156.0,156.0
3,"(/home/workspace/lib/valueflow.cpp, _Z12lifetimeTypeB5cxx11PK5TokenPKN9ValueFlow5ValueE, 2834)",0,1,84,0,1.0,1.0,156.0,156.0
4,"(/home/workspace/lib/checkautovariables.cpp, _ZL21variableIsUsedInScopePK5TokeniPK5Scope, 200)",0,1,84,0,1.0,1.0,156.0,156.0


# Method level SBFL

In [7]:
spectra["file"] = spectra.index.map(lambda x: x[0])
spectra["func"] = spectra.index.map(lambda x: x[1])
spectra["line"] = spectra.index.map(lambda x: x[2])
# (file, func) is meth
spectra["meth"] = spectra.apply(lambda row: (row.file, row.func), axis=1)
spectra.reset_index(drop=True, inplace=True)
display(spectra)

,line,e_p,e_f,n_p,n_f,Sop2,Sochiai,Rop2,Rochiai,file,func,meth
0,44,18,0,66,1,-0.211765,0.000000,36411.0,39954.0,/home/workspace/lib/checkunusedvar.h,_ZN14CheckUnusedVarD2Ev,"(/home/workspace/lib/checkunusedvar.h, _ZN14CheckUnusedVarD2Ev)"
1,47,84,1,0,0,0.011765,0.108465,12357.0,12357.0,/home/workspace/lib/checkunusedvar.h,_ZN14CheckUnusedVarC2Ev,"(/home/workspace/lib/checkunusedvar.h, _ZN14CheckUnusedVarC2Ev)"
2,48,84,1,0,0,0.011765,0.108465,12357.0,12357.0,/home/workspace/lib/checkunusedvar.h,_ZN14CheckUnusedVarC2Ev,"(/home/workspace/lib/checkunusedvar.h, _ZN14CheckUnusedVarC2Ev)"
3,51,18,0,66,1,-0.211765,0.000000,36411.0,39954.0,/home/workspace/lib/checkunusedvar.h,_ZN14CheckUnusedVarC2EPK9TokenizerPK8SettingsP11ErrorLogger,"(/home/workspace/lib/checkunusedvar.h, _ZN14CheckUnusedVarC2EPK9TokenizerPK8SettingsP11ErrorLogger)"
4,52,18,0,66,1,-0.211765,0.000000,36411.0,39954.0,/home/workspace/lib/checkunusedvar.h,_ZN14CheckUnusedVarC2EPK9TokenizerPK8SettingsP11ErrorLogger,"(/home/workspace/lib/checkunusedvar.h, _ZN14CheckUnusedVarC2EPK9TokenizerPK8SettingsP11ErrorLogger)"
...,...,...,...,...,...,...,...,...,...,...,...,...
39949,364,61,1,23,0,0.282353,0.127000,8250.0,8250.0,/usr/include/c++/7/bits/refwrap.h,_ZSt3refIKNSt7__debug13unordered_mapIiPK8VariableSt4hashIiESt8equal_toIiESaISt4pairIKiS4_EEEEESt17reference_wrapperIT_ERSG_,"(/usr/include/c++/7/bits/refwrap.h, _ZSt3refIKNSt7__debug13unordered_mapIiPK8VariableSt4hashIiESt8equal_toIiESaISt4pairIKiS4_EEEEESt17reference_wrapperIT_ERSG_)"
39950,365,61,1,23,0,0.282353,0.127000,8250.0,8250.0,/usr/include/c++/7/bits/refwrap.h,_ZSt3refIKNSt7__debug13unordered_mapIiPK8VariableSt4hashIiESt8equal_toIiESaISt4pairIKiS4_EEEEESt17reference_wrapperIT_ERSG_,"(/usr/include/c++/7/bits/refwrap.h, _ZSt3refIKNSt7__debug13unordered_mapIiPK8VariableSt4hashIiESt8equal_toIiESaISt4pairIKiS4_EEEEESt17reference_wrapperIT_ERSG_)"
39951,166,17,0,67,1,-0.200000,0.000000,35684.0,39954.0,/home/workspace/lib/templatesimplifier.h,_ZN18TemplateSimplifier12TokenAndName5tokenEP5Token,"(/home/workspace/lib/templatesimplifier.h, _ZN18TemplateSimplifier12TokenAndName5tokenEP5Token)"
39952,167,17,0,67,1,-0.200000,0.000000,35684.0,39954.0,/home/workspace/lib/templatesimplifier.h,_ZN18TemplateSimplifier12TokenAndName5tokenEP5Token,"(/home/workspace/lib/templatesimplifier.h, _ZN18TemplateSimplifier12TokenAndName5tokenEP5Token)"


## Max aggregation

In [8]:
print("[Op2]")
spectra_max_op2 = spectra.copy().sort_values(
    ["Sop2", "meth", "line"], ascending=False
)
spectra_max_op2.drop_duplicates(subset="meth", keep="first", inplace=True)
spectra_max_op2 = spectra_max_op2[
    ["meth", "line", "e_p", "e_f", "n_p", "n_f", "Sop2"]
]
spectra_max_op2["Rop2"] = spectra_max_op2.Sop2.rank(
    ascending=False, method="max"
)
print("Faulty method:")
display(
    spectra_max_op2.loc[spectra_max_op2.meth.isin(fault_meths)].reset_index(
        drop=True
    )
)
print("Top 20 methods:")
display(spectra_max_op2.head(20).reset_index(drop=True))
print()
print("[Ochiai]")
spectra_max_ochiai = spectra.copy().sort_values(
    ["Sochiai", "meth", "line"], ascending=False
)
spectra_max_ochiai.drop_duplicates(subset="meth", keep="first", inplace=True)
spectra_max_ochiai = spectra_max_ochiai[
    ["meth", "line", "e_p", "e_f", "n_p", "n_f", "Sochiai"]
]
spectra_max_ochiai["Rochiai"] = spectra_max_ochiai.Sochiai.rank(
    ascending=False, method="max"
)
print("Faulty method:")
display(
    spectra_max_ochiai.loc[
        spectra_max_ochiai.meth.isin(fault_meths)
    ].reset_index(drop=True)
)
print("Top 20 methods:")
display(spectra_max_ochiai.head(20).reset_index(drop=True))


[Op2]
Faulty method:


,meth,line,e_p,e_f,n_p,n_f,Sop2,Rop2
0,"(/home/workspace/lib/valueflow.cpp, _ZL18isLifetimeBorrowedPK9ValueTypeS1_)",3073,4,1,80,0,0.952941,118.0


Top 20 methods:


,meth,line,e_p,e_f,n_p,n_f,Sop2,Rop2
0,"(/home/workspace/lib/valueflow.cpp, _ZZNK13LifetimeStore11byDerefCopyEP5TokenP9TokenListP11ErrorLoggerPK8SettingsENKUlPKS0_E_clESA_)",3433,0,1,84,0,1.0,29.0
1,"(/home/workspace/lib/valueflow.cpp, _ZZL17valueFlowLifetimeP9TokenListP14SymbolDatabaseP11ErrorLoggerPK8SettingsENKUlPK5TokenE_clESA_)",3657,0,1,84,0,1.0,29.0
2,"(/home/workspace/lib/valueflow.cpp, _ZNK29MultiValueFlowForwardAnalyzer11updateScopeEPK5Tokenb)",4856,0,1,84,0,1.0,29.0
3,"(/home/workspace/lib/valueflow.cpp, _ZNK13LifetimeStore5byValIZNKS_5byValEP5TokenP9TokenListP11ErrorLoggerPK8SettingsEUlPKS1_E_EEvS2_S4_S6_S9_T_)",3363,0,1,84,0,1.0,29.0
4,"(/home/workspace/lib/valueflow.cpp, _ZNK13LifetimeStore11byDerefCopyIZNKS_11byDerefCopyEP5TokenP9TokenListP11ErrorLoggerPK8SettingsEUlPKS1_E_EEvS2_S4_S6_S9_T_)",3424,0,1,84,0,1.0,29.0
5,"(/home/workspace/lib/valueflow.cpp, _ZL28valueFlowLifetimeConstructorP5TokenPK4TypeP9TokenListP11ErrorLoggerPK8Settings)",3544,0,1,84,0,1.0,29.0
6,"(/home/workspace/lib/valueflow.cpp, _ZL28valueFlowLifetimeConstructorP5TokenP9TokenListP11ErrorLoggerPK8Settings)",3581,0,1,84,0,1.0,29.0
7,"(/home/workspace/lib/valueflow.cpp, _ZL25valueFlowLifetimeFunctionP5TokenP9TokenListP11ErrorLoggerPK8Settings)",3513,0,1,84,0,1.0,29.0
8,"(/home/workspace/lib/valueflow.cpp, _ZL24valueFlowForwardLifetimeP5TokenP9TokenListP11ErrorLoggerPK8Settings)",3187,0,1,84,0,1.0,29.0
9,"(/home/workspace/lib/valueflow.cpp, _ZL15isLifetimeOwnedPK9ValueTypeS1_)",3048,0,1,84,0,1.0,29.0



[Ochiai]
Faulty method:


,meth,line,e_p,e_f,n_p,n_f,Sochiai,Rochiai
0,"(/home/workspace/lib/valueflow.cpp, _ZL18isLifetimeBorrowedPK9ValueTypeS1_)",3073,4,1,80,0,0.447214,118.0


Top 20 methods:


,meth,line,e_p,e_f,n_p,n_f,Sochiai,Rochiai
0,"(/home/workspace/lib/valueflow.cpp, _ZZNK13LifetimeStore11byDerefCopyEP5TokenP9TokenListP11ErrorLoggerPK8SettingsENKUlPKS0_E_clESA_)",3433,0,1,84,0,1.0,29.0
1,"(/home/workspace/lib/valueflow.cpp, _ZZL17valueFlowLifetimeP9TokenListP14SymbolDatabaseP11ErrorLoggerPK8SettingsENKUlPK5TokenE_clESA_)",3657,0,1,84,0,1.0,29.0
2,"(/home/workspace/lib/valueflow.cpp, _ZNK29MultiValueFlowForwardAnalyzer11updateScopeEPK5Tokenb)",4856,0,1,84,0,1.0,29.0
3,"(/home/workspace/lib/valueflow.cpp, _ZNK13LifetimeStore5byValIZNKS_5byValEP5TokenP9TokenListP11ErrorLoggerPK8SettingsEUlPKS1_E_EEvS2_S4_S6_S9_T_)",3363,0,1,84,0,1.0,29.0
4,"(/home/workspace/lib/valueflow.cpp, _ZNK13LifetimeStore11byDerefCopyIZNKS_11byDerefCopyEP5TokenP9TokenListP11ErrorLoggerPK8SettingsEUlPKS1_E_EEvS2_S4_S6_S9_T_)",3424,0,1,84,0,1.0,29.0
5,"(/home/workspace/lib/valueflow.cpp, _ZL28valueFlowLifetimeConstructorP5TokenPK4TypeP9TokenListP11ErrorLoggerPK8Settings)",3544,0,1,84,0,1.0,29.0
6,"(/home/workspace/lib/valueflow.cpp, _ZL28valueFlowLifetimeConstructorP5TokenP9TokenListP11ErrorLoggerPK8Settings)",3581,0,1,84,0,1.0,29.0
7,"(/home/workspace/lib/valueflow.cpp, _ZL25valueFlowLifetimeFunctionP5TokenP9TokenListP11ErrorLoggerPK8Settings)",3513,0,1,84,0,1.0,29.0
8,"(/home/workspace/lib/valueflow.cpp, _ZL24valueFlowForwardLifetimeP5TokenP9TokenListP11ErrorLoggerPK8Settings)",3187,0,1,84,0,1.0,29.0
9,"(/home/workspace/lib/valueflow.cpp, _ZL15isLifetimeOwnedPK9ValueTypeS1_)",3048,0,1,84,0,1.0,29.0


## Line2Method voting (weight: sbfl score / dense rank)

In [9]:
print("[Op2]")
ballot_op2 = {}
spectra_max_op2 = spectra.copy().sort_values(
    ["Sop2", "meth", "line"], ascending=False
)
spectra_max_op2["Rop2D"] = spectra_max_op2.Sop2.rank(
    ascending=False, method="dense"
)
spectra_max_op2["vote"] = spectra_max_op2.Sop2 / spectra_max_op2.Rop2D
for row in spectra_max_op2.itertuples():
    if row.meth not in ballot_op2:
        ballot_op2[row.meth] = row.vote
    else:
        ballot_op2[row.meth] += row.vote
L2M_op2 = pd.DataFrame(
    {"meth": list(ballot_op2.keys()), "vote": list(ballot_op2.values())}
)
L2M_op2["rank"] = L2M_op2.vote.rank(ascending=False, method="max")
L2M_op2.sort_values("rank", inplace=True)
L2M_op2.reset_index(drop=True, inplace=True)
for fault_meth in fault_meths:
    print("Faulty method:")
    display(L2M_op2.loc[L2M_op2.meth == fault_meth].reset_index(drop=True))
    print("Voters:")
    display(
        spectra_max_op2.loc[spectra_max_op2.meth == fault_meth].reset_index(
            drop=True
        )
    )
print("Top 5 methods:")
display(L2M_op2.head(5).reset_index(drop=True))
for idx, top_meth in enumerate(L2M_op2.head(5).meth.tolist(), 1):
    print(f"Top {idx}:")
    display(L2M_op2.loc[L2M_op2.meth == top_meth].reset_index(drop=True))
    print("Voters:")
    display(
        spectra_max_op2.loc[spectra_max_op2.meth == top_meth].reset_index(
            drop=True
        )
    )
print()
print("[Ochiai]")
ballot_ochiai = {}
spectra_max_ochiai = spectra.copy().sort_values(
    ["Sochiai", "meth", "line"], ascending=False
)
spectra_max_ochiai["RochiaiD"] = spectra_max_ochiai.Sochiai.rank(
    ascending=False, method="dense"
)
spectra_max_ochiai["vote"] = (
    spectra_max_ochiai.Sochiai / spectra_max_ochiai.RochiaiD
)
for row in spectra_max_ochiai.itertuples():
    if row.meth not in ballot_ochiai:
        ballot_ochiai[row.meth] = row.vote
    else:
        ballot_ochiai[row.meth] += row.vote
L2M_ochiai = pd.DataFrame(
    {"meth": list(ballot_ochiai.keys()), "vote": list(ballot_ochiai.values())}
)
L2M_ochiai["rank"] = L2M_ochiai.vote.rank(ascending=False, method="max")
L2M_ochiai.sort_values("rank", inplace=True)
L2M_ochiai.reset_index(drop=True, inplace=True)
for fault_meth in fault_meths:
    print("Faulty method:")
    display(
        L2M_ochiai.loc[L2M_ochiai.meth == fault_meth].reset_index(drop=True)
    )
    print("Voters:")
    display(
        spectra_max_ochiai.loc[
            spectra_max_ochiai.meth == fault_meth
        ].reset_index(drop=True)
    )
print("Top 5 methods:")
display(L2M_ochiai.head(5).reset_index(drop=True))
for idx, top_meth in enumerate(L2M_ochiai.head(5).meth.tolist(), 1):
    print(f"Top {idx}:")
    display(L2M_ochiai.loc[L2M_ochiai.meth == top_meth].reset_index(drop=True))
    print("Voters:")
    display(
        spectra_max_ochiai.loc[spectra_max_ochiai.meth == top_meth].reset_index(
            drop=True
        )
    )


[Op2]
Faulty method:


,meth,vote,rank
0,"(/home/workspace/lib/valueflow.cpp, _ZL18isLifetimeBorrowedPK9ValueTypeS1_)",0.714088,147.0


Voters:


,line,e_p,e_f,n_p,n_f,Sop2,Sochiai,Rop2,Rochiai,file,func,meth,Rop2D,vote
0,3073,4,1,80,0,0.952941,0.447214,656.0,656.0,/home/workspace/lib/valueflow.cpp,_ZL18isLifetimeBorrowedPK9ValueTypeS1_,"(/home/workspace/lib/valueflow.cpp, _ZL18isLifetimeBorrowedPK9ValueTypeS1_)",5.0,0.190588
1,3080,6,1,78,0,0.929412,0.377964,848.0,848.0,/home/workspace/lib/valueflow.cpp,_ZL18isLifetimeBorrowedPK9ValueTypeS1_,"(/home/workspace/lib/valueflow.cpp, _ZL18isLifetimeBorrowedPK9ValueTypeS1_)",7.0,0.132773
2,3075,13,1,71,0,0.847059,0.267261,1535.0,1535.0,/home/workspace/lib/valueflow.cpp,_ZL18isLifetimeBorrowedPK9ValueTypeS1_,"(/home/workspace/lib/valueflow.cpp, _ZL18isLifetimeBorrowedPK9ValueTypeS1_)",14.0,0.060504
3,3070,18,1,66,0,0.788235,0.229416,2099.0,2099.0,/home/workspace/lib/valueflow.cpp,_ZL18isLifetimeBorrowedPK9ValueTypeS1_,"(/home/workspace/lib/valueflow.cpp, _ZL18isLifetimeBorrowedPK9ValueTypeS1_)",19.0,0.041486
4,3068,20,1,64,0,0.764706,0.218218,2330.0,2330.0,/home/workspace/lib/valueflow.cpp,_ZL18isLifetimeBorrowedPK9ValueTypeS1_,"(/home/workspace/lib/valueflow.cpp, _ZL18isLifetimeBorrowedPK9ValueTypeS1_)",21.0,0.036415
5,3083,22,1,62,0,0.741176,0.208514,2614.0,2614.0,/home/workspace/lib/valueflow.cpp,_ZL18isLifetimeBorrowedPK9ValueTypeS1_,"(/home/workspace/lib/valueflow.cpp, _ZL18isLifetimeBorrowedPK9ValueTypeS1_)",23.0,0.032225
6,3077,22,1,62,0,0.741176,0.208514,2614.0,2614.0,/home/workspace/lib/valueflow.cpp,_ZL18isLifetimeBorrowedPK9ValueTypeS1_,"(/home/workspace/lib/valueflow.cpp, _ZL18isLifetimeBorrowedPK9ValueTypeS1_)",23.0,0.032225
7,3078,23,1,61,0,0.729412,0.204124,2814.0,2814.0,/home/workspace/lib/valueflow.cpp,_ZL18isLifetimeBorrowedPK9ValueTypeS1_,"(/home/workspace/lib/valueflow.cpp, _ZL18isLifetimeBorrowedPK9ValueTypeS1_)",24.0,0.030392
8,3076,26,1,58,0,0.694118,0.192450,3240.0,3240.0,/home/workspace/lib/valueflow.cpp,_ZL18isLifetimeBorrowedPK9ValueTypeS1_,"(/home/workspace/lib/valueflow.cpp, _ZL18isLifetimeBorrowedPK9ValueTypeS1_)",27.0,0.025708
9,3074,27,1,57,0,0.682353,0.188982,3601.0,3601.0,/home/workspace/lib/valueflow.cpp,_ZL18isLifetimeBorrowedPK9ValueTypeS1_,"(/home/workspace/lib/valueflow.cpp, _ZL18isLifetimeBorrowedPK9ValueTypeS1_)",28.0,0.024370


Top 5 methods:


...

,line,e_p,e_f,n_p,n_f,Sop2,Sochiai,Rop2,Rochiai,file,func,meth,RochiaiD,vote
0,274,0,1,84,0,1.000000,1.000000,156.0,156.0,/home/workspace/lib/astutils.cpp,_Z11isTemporarybPK5TokenPK7Libraryb,"(/home/workspace/lib/astutils.cpp, _Z11isTemporarybPK5TokenPK7Libraryb)",1.0,1.000000
1,273,0,1,84,0,1.000000,1.000000,156.0,156.0,/home/workspace/lib/astutils.cpp,_Z11isTemporarybPK5TokenPK7Libraryb,"(/home/workspace/lib/astutils.cpp, _Z11isTemporarybPK5TokenPK7Libraryb)",1.0,1.000000
2,272,0,1,84,0,1.000000,1.000000,156.0,156.0,/home/workspace/lib/astutils.cpp,_Z11isTemporarybPK5TokenPK7Libraryb,"(/home/workspace/lib/astutils.cpp, _Z11isTemporarybPK5TokenPK7Libraryb)",1.0,1.000000
3,269,0,1,84,0,1.000000,1.000000,156.0,156.0,/home/workspace/lib/astutils.cpp,_Z11isTemporarybPK5TokenPK7Libraryb,"(/home/workspace/lib/astutils.cpp, _Z11isTemporarybPK5TokenPK7Libraryb)",1.0,1.000000
4,264,0,1,84,0,1.000000,1.000000,156.0,156.0,/home/workspace/lib/astutils.cpp,_Z11isTemporarybPK5TokenPK7Libraryb,"(/home/workspace/lib/astutils.cpp, _Z11isTemporarybPK5TokenPK7Libraryb)",1.0,1.000000
5,263,0,1,84,0,1.000000,1.000000,156.0,156.0,/home/workspace/lib/astutils.cpp,_Z11isTemporarybPK5TokenPK7Libraryb,"(/home/workspace/lib/astutils.cpp, _Z11isTemporarybPK5TokenPK7Libraryb)",1.0,1.000000
6,255,0,1,84,0,1.000000,1.000000,156.0,156.0,/home/workspace/lib/astutils.cpp,_Z11isTemporarybPK5TokenPK7Libraryb,"(/home/workspace/lib/astutils.cpp, _Z11isTemporarybPK5TokenPK7Libraryb)",1.0,1.000000
7,251,0,1,84,0,1.000000,1.000000,156.0,156.0,/home/workspace/lib/astutils.cpp,_Z11isTemporarybPK5TokenPK7Libraryb,"(/home/workspace/lib/astutils.cpp, _Z11isTemporarybPK5TokenPK7Libraryb)",1.0,1.000000
8,249,0,1,84,0,1.000000,1.000000,156.0,156.0,/home/workspace/lib/astutils.cpp,_Z11isTemporarybPK5TokenPK7Libraryb,"(/home/workspace/lib/astutils.cpp, _Z11isTemporarybPK5TokenPK7Libraryb)",1.0,1.000000
9,245,0,1,84,0,1.000000,1.000000,156.0,156.0,/home/workspace/lib/astutils.cpp,_Z11isTemporarybPK5TokenPK7Libraryb,"(/home/workspace/lib/astutils.cpp, _Z11isTemporarybPK5TokenPK7Libraryb)",1.0,1.000000


Top 5:


,meth,vote,rank
0,"(/home/workspace/lib/valueflow.cpp, _ZNK13LifetimeStore11byDerefCopyIZNKS_11byDerefCopyEP5TokenP9TokenListP11ErrorLoggerPK8SettingsEUlPKS1_E_EEvS2_S4_S6_S9_T_)",11.414214,5.0


Voters:


,line,e_p,e_f,n_p,n_f,Sop2,Sochiai,Rop2,Rochiai,file,func,meth,RochiaiD,vote
0,3424,0,1,84,0,1.000000,1.000000,156.0,156.0,/home/workspace/lib/valueflow.cpp,_ZNK13LifetimeStore11byDerefCopyIZNKS_11byDerefCopyEP5TokenP9TokenListP11ErrorLoggerPK8SettingsEUlPKS1_E_EEvS2_S4_S6_S9_T_,"(/home/workspace/lib/valueflow.cpp, _ZNK13LifetimeStore11byDerefCopyIZNKS_11byDerefCopyEP5TokenP9TokenListP11ErrorLoggerPK8SettingsEUlPKS1_E_EEvS2_S4_S6_S9_T_)",1.0,1.000000
1,3423,0,1,84,0,1.000000,1.000000,156.0,156.0,/home/workspace/lib/valueflow.cpp,_ZNK13LifetimeStore11byDerefCopyIZNKS_11byDerefCopyEP5TokenP9TokenListP11ErrorLoggerPK8SettingsEUlPKS1_E_EEvS2_S4_S6_S9_T_,"(/home/workspace/lib/valueflow.cpp, _ZNK13LifetimeStore11byDerefCopyIZNKS_11byDerefCopyEP5TokenP9TokenListP11ErrorLoggerPK8SettingsEUlPKS1_E_EEvS2_S4_S6_S9_T_)",1.0,1.000000
2,3422,0,1,84,0,1.000000,1.000000,156.0,156.0,/home/workspace/lib/valueflow.cpp,_ZNK13LifetimeStore11byDerefCopyIZNKS_11byDerefCopyEP5TokenP9TokenListP11ErrorLoggerPK8SettingsEUlPKS1_E_EEvS2_S4_S6_S9_T_,"(/home/workspace/lib/valueflow.cpp, _ZNK13LifetimeStore11byDerefCopyIZNKS_11byDerefCopyEP5TokenP9TokenListP11ErrorLoggerPK8SettingsEUlPKS1_E_EEvS2_S4_S6_S9_T_)",1.0,1.000000
3,3421,0,1,84,0,1.000000,1.000000,156.0,156.0,/home/workspace/lib/valueflow.cpp,_ZNK13LifetimeStore11byDerefCopyIZNKS_11byDerefCopyEP5TokenP9TokenListP11ErrorLoggerPK8SettingsEUlPKS1_E_EEvS2_S4_S6_S9_T_,"(/home/workspace/lib/valueflow.cpp, _ZNK13LifetimeStore11byDerefCopyIZNKS_11byDerefCopyEP5TokenP9TokenListP11ErrorLoggerPK8SettingsEUlPKS1_E_EEvS2_S4_S6_S9_T_)",1.0,1.000000
4,3419,0,1,84,0,1.000000,1.000000,156.0,156.0,/home/workspace/lib/valueflow.cpp,_ZNK13LifetimeStore11byDerefCopyIZNKS_11byDerefCopyEP5TokenP9TokenListP11ErrorLoggerPK8SettingsEUlPKS1_E_EEvS2_S4_S6_S9_T_,"(/home/workspace/lib/valueflow.cpp, _ZNK13LifetimeStore11byDerefCopyIZNKS_11byDerefCopyEP5TokenP9TokenListP11ErrorLoggerPK8SettingsEUlPKS1_E_EEvS2_S4_S6_S9_T_)",1.0,1.000000
5,3418,0,1,84,0,1.000000,1.000000,156.0,156.0,/home/workspace/lib/valueflow.cpp,_ZNK13LifetimeStore11byDerefCopyIZNKS_11byDerefCopyEP5TokenP9TokenListP11ErrorLoggerPK8SettingsEUlPKS1_E_EEvS2_S4_S6_S9_T_,"(/home/workspace/lib/valueflow.cpp, _ZNK13LifetimeStore11byDerefCopyIZNKS_11byDerefCopyEP5TokenP9TokenListP11ErrorLoggerPK8SettingsEUlPKS1_E_EEvS2_S4_S6_S9_T_)",1.0,1.000000
6,3417,0,1,84,0,1.000000,1.000000,156.0,156.0,/home/workspace/lib/valueflow.cpp,_ZNK13LifetimeStore11byDerefCopyIZNKS_11byDerefCopyEP5TokenP9TokenListP11ErrorLoggerPK8SettingsEUlPKS1_E_EEvS2_S4_S6_S9_T_,"(/home/workspace/lib/valueflow.cpp, _ZNK13LifetimeStore11byDerefCopyIZNKS_11byDerefCopyEP5TokenP9TokenListP11ErrorLoggerPK8SettingsEUlPKS1_E_EEvS2_S4_S6_S9_T_)",1.0,1.000000
7,3416,0,1,84,0,1.000000,1.000000,156.0,156.0,/home/workspace/lib/valueflow.cpp,_ZNK13LifetimeStore11byDerefCopyIZNKS_11byDerefCopyEP5TokenP9TokenListP11ErrorLoggerPK8SettingsEUlPKS1_E_EEvS2_S4_S6_S9_T_,"(/home/workspace/lib/valueflow.cpp, _ZNK13LifetimeStore11byDerefCopyIZNKS_11byDerefCopyEP5TokenP9TokenListP11ErrorLoggerPK8SettingsEUlPKS1_E_EEvS2_S4_S6_S9_T_)",1.0,1.000000
8,3415,0,1,84,0,1.000000,1.000000,156.0,156.0,/home/workspace/lib/valueflow.cpp,_ZNK13LifetimeStore11byDerefCopyIZNKS_11byDerefCopyEP5TokenP9TokenListP11ErrorLoggerPK8SettingsEUlPKS1_E_EEvS2_S4_S6_S9_T_,"(/home/workspace/lib/valueflow.cpp, _ZNK13LifetimeStore11byDerefCopyIZNKS_11byDerefCopyEP5TokenP9TokenListP11ErrorLoggerPK8SettingsEUlPKS1_E_EEvS2_S4_S6_S9_T_)",1.0,1.000000
9,3413,0,1,84,0,1.000000,1.000000,156.0,156.0,/home/workspace/lib/valueflow.cpp,_ZNK13LifetimeStore11byDerefCopyIZNKS_11byDerefCopyEP5TokenP9TokenListP11ErrorLoggerPK8SettingsEUlPKS1_E_EEvS2_S4_S6_S9_T_,"(/home/workspace/lib/valueflow.cpp, _ZNK13LifetimeStore11byDerefCopyIZNKS_11byDerefCopyEP5TokenP9TokenListP11ErrorLoggerPK8SettingsEUlPKS1_E_EEvS2_S4_S6_S9_T_)",1.0,1.000000


# 감상

단순히 passing test에 의해서 한 번도 실행되지 않고 failing test에 의해서 실행되는 method가 너무 많다 (sbfl max agg top max rank: 29)